In [1]:
import json
from collections.abc import Mapping
from typing import Any

from jsonschema import Draft202012Validator, FormatChecker
from pydantic import BaseModel

JSONLike = Any | str | BaseModel  # keep it simple: one alias for both inputs


def _coerce(value: JSONLike, *, as_schema: bool) -> Any:  # noqa: ANN401
    """
    Coerce input to a Python object suitable for validation.
    - If BaseModel: dump to schema (for schema) or to data (for data).
    - If str: json.loads.
    - Otherwise: pass through.
    Additionally, if as_schema=True, ensure result is a mapping.
    """
    if isinstance(value, BaseModel):
        value = value.__class__.model_json_schema() if as_schema else value.model_dump()
    elif isinstance(value, str):
        value = json.loads(value)

    if as_schema and not isinstance(value, Mapping):
        raise TypeError("Schema must be a JSON object (mapping) after coercion.")
    return value


def validate_json_schema(data: JSONLike, schema: JSONLike) -> tuple[bool, str | None]:
    """
    Validate `data` against JSON `schema`.

    Returns (is_valid, error_report). If invalid, error_report lists all issues.
    """
    schema = _coerce(schema, as_schema=True)
    data = _coerce(data, as_schema=False)

    # Draft202012Validator is the implementation of the JSON Schema 2020-12 draft, which is the
    # latest stable version of the standard.
    validator = Draft202012Validator(schema, format_checker=FormatChecker())
    errors = sorted(validator.iter_errors(data), key=lambda e: list(e.path))

    if not errors:
        return True, None

    validation_errors = []
    for err in errors:
        path_elems = [str(p) for p in err.path]
        path = "/" + "/".join(path_elems) if path_elems else "(root)"
        validation_errors.append(f"{path}: {err.message}")
    return False, validation_errors

schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age":  {"type": "integer", "minimum": 0, "maximum": 120},
    },
    "required": ["name", "age"],
    "additionalProperties": False,
}

# data = {"name": "Alice", "age": -1}  # violates minimum 0
data = {'age': -1} # violates required name and minimum age

valid, validation_errors = validate_json_schema(data, schema)
print("Valid:", valid)
print("Validation Errors:")
for err in (validation_errors or []):
    print(err)

Valid: False
Validation Errors:
(root): 'name' is a required property
/age: -1 is less than the minimum of 0
